In [47]:
import pandas as pd
import numpy as np 
from surprise import SVD
from surprise import KNNBaseline
from surprise.model_selection import train_test_split
from surprise.model_selection import LeaveOneOut
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from collections import defaultdict

### 数据在此下载 ###
https://github.com/SophonPlus/ChineseNlpCorpus/blob/master/datasets/ez_douban/intro.ipynb

In [2]:
movies = pd.read_csv( './data/douban/movies.csv')
ratings = pd.read_csv('./data/douban/ratings.csv')

combine_movie_rating= pd.merge(ratings,movies,on='movieId',how='inner')
combine_movie_rating=combine_movie_rating.drop(['timestamp'],axis = 1)
combine_movie_rating = combine_movie_rating.dropna(axis = 0 ,subset=['title'])
print('评分数量：%d' % len(combine_movie_rating))
combine_movie_rating.sample(10)

评分数量：2604995


,userId,movieId,rating,title
1917978,7610,4730,5,Nineteen Eighty-Four
1377448,7216,1884,4,Gone with the Wind
288771,8396,298,5,The Big Bang Theory
2775115,2507,33878,5,罪域
1156923,88,1453,2,秋喜
927004,1704,1050,5,August Rush
1110609,7210,1368,3,神话
2522138,3775,12827,3,Twist Calm And Passion
2546244,1295,13754,4,The West Wing
179521,8455,156,3,Up


In [3]:
movie_rating_count=pd.DataFrame(combine_movie_rating.
                    groupby(['movieId'])['rating'].
                    count().
                    reset_index().
                    rename(columns={'rating':'totalRatingCount'})                   
                   )
rating_with_totalRatingCount = combine_movie_rating.merge(movie_rating_count,left_on='movieId',right_on='movieId')
rating_with_totalRatingCount.head()

,userId,movieId,rating,title,totalRatingCount
0,0,1,4,Harry Potter and the Deathly Hallows: Part II,1703
1,21,1,4,Harry Potter and the Deathly Hallows: Part II,1703
2,25,1,5,Harry Potter and the Deathly Hallows: Part II,1703
3,34,1,4,Harry Potter and the Deathly Hallows: Part II,1703
4,36,1,5,Harry Potter and the Deathly Hallows: Part II,1703


In [4]:
rating_with_totalRatingCount['totalRatingCount'].quantile(np.arange(.9,1,.01))

0.90    2351.0
0.91    2441.0
0.92    2654.0
0.93    2814.0
0.94    2958.0
0.95    3062.0
0.96    3330.0
0.97    3731.0
0.98    4432.0
0.99    5072.0
Name: totalRatingCount, dtype: float64

In [5]:
#取10%的最热门的电影
popular_threshold=2351 
rating_with_totalRatingCount = combine_movie_rating.merge(movie_rating_count,left_on='movieId',right_on='movieId')
popular_movies_rating= rating_with_totalRatingCount.query('totalRatingCount>=@popular_threshold')
print('热门电影数据量：%d' % len(popular_movies_rating))

热门电影数据量：262455


In [55]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(popular_movies_rating[['userId', 'movieId', 'rating']], reader)
train, test = train_test_split(data, test_size=.25, random_state=0)
svd_model = SVD(random_state=0)
svd_model.fit(train)
predict = svd_model.test(test)   
print("RMSE: ",accuracy.mae(predict, verbose=False))
print("MAE: ",accuracy.rmse(predict, verbose=False))
predict[:10]

RMSE:  0.5501276131831262
MAE:  0.6968515836030541


[Prediction(uid=1426, iid=1130, r_ui=5.0, est=3.6235014684425333, details={'was_impossible': False}),
 Prediction(uid=9899, iid=609, r_ui=4.0, est=4.427492924992114, details={'was_impossible': False}),
 Prediction(uid=10837, iid=150, r_ui=5.0, est=4.827915792544979, details={'was_impossible': False}),
 Prediction(uid=4643, iid=995, r_ui=4.0, est=4.211769405268239, details={'was_impossible': False}),
 Prediction(uid=9769, iid=247, r_ui=5.0, est=4.254900765324765, details={'was_impossible': False}),
 Prediction(uid=5582, iid=170, r_ui=4.0, est=4.31240196218265, details={'was_impossible': False}),
 Prediction(uid=6605, iid=156, r_ui=5.0, est=4.245027154626259, details={'was_impossible': False}),
 Prediction(uid=9884, iid=21, r_ui=4.0, est=3.824987946672277, details={'was_impossible': False}),
 Prediction(uid=17492, iid=96, r_ui=4.0, est=4.72638009370838, details={'was_impossible': False}),
 Prediction(uid=25829, iid=738, r_ui=5.0, est=4.788603891699279, details={'was_impossible': False})]

In [56]:
def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
# trainset = data.build_full_trainset()
# svd_model.fit(trainset)

#从训练集中创建一个测试集,该测试集中包含了所有用户没有看过的电影，即该测试集中的数据不在训练集中
# testset = trainset.build_anti_testset()
# predictions = svd_model.test(testset)

# top_n = get_top_n(predictions, n=8)

# for uid, user_ratings in top_n.items():
#      print(uid, [(iid,round(rating,1)) for (iid, rating) in user_ratings])

In [72]:
#定义留一法交叉验证,在测试集中每个用户只保留一条评分记录
LOOCV = LeaveOneOut(n_splits=1, random_state=1)

for trainSet, testSet in LOOCV.split(data):
    
    #在训练集上训练模型
    svd_model.fit(trainSet)
    #在测试集上预测
    leftOutPredictions = svd_model.test(testSet)
    
    #从训练集中创建一个测试集,该测试集中包含了所有用户没有看过的电影，即该测试集中的数据不在训练集中
    bigTestSet = trainSet.build_anti_testset()
    
    #得到所有用户没有看过的所有电影的预测评分
    allPredictions = svd_model.test(bigTestSet)
    
    #从每个用户的未看过的电影的预测评分中抽取前10个得分最高的电影
    topNPredicted = get_top_n(allPredictions, n=10)

#打印为每个用户推荐的10部电影和对它们的评分
for uid, user_ratings in topNPredicted.items():
      print(uid, [(iid,round(rating,1)) for (iid, rating) in user_ratings])

0 [(414, 5), (415, 5), (738, 5), (750, 5), (1011, 5.0), (695, 5.0), (311, 4.9), (745, 4.9), (737, 4.9), (1008, 4.9)]
2 [(20, 4.8), (998, 4.8), (745, 4.8), (737, 4.8), (92, 4.7), (695, 4.7), (1007, 4.7), (1011, 4.7), (151, 4.7), (738, 4.6)]
3 [(19, 5), (150, 5), (414, 5), (92, 5.0), (311, 5.0), (20, 5.0), (737, 4.9), (745, 4.9), (750, 4.9), (151, 4.8)]
4 [(414, 4.5), (20, 4.5), (92, 4.5), (124, 4.5), (750, 4.5), (745, 4.5), (311, 4.5), (737, 4.4), (993, 4.4), (738, 4.4)]
5 [(1011, 4.4), (20, 4.4), (415, 4.4), (124, 4.3), (737, 4.3), (15, 4.3), (311, 4.3), (98, 4.2), (93, 4.2), (405, 4.2)]
6 [(150, 4.7), (415, 4.7), (20, 4.7), (311, 4.6), (92, 4.6), (414, 4.6), (745, 4.6), (124, 4.6), (405, 4.6), (750, 4.5)]
9 [(716, 4.8), (20, 4.7), (376, 4.6), (377, 4.6), (182, 4.6), (238, 4.5), (98, 4.4), (1446, 4.4), (679, 4.4), (96, 4.3)]
10 [(20, 5), (737, 4.9), (15, 4.8), (695, 4.8), (19, 4.8), (1007, 4.8), (405, 4.7), (96, 4.7), (182, 4.7), (238, 4.7)]
11 [(20, 4.6), (745, 4.5), (182, 4.5), (750,

547 [(405, 4.8), (679, 4.8), (182, 4.7), (997, 4.7), (15, 4.7), (377, 4.7), (979, 4.6), (609, 4.5), (998, 4.5), (971, 4.5)]
548 [(20, 4.5), (150, 4.5), (414, 4.5), (92, 4.5), (738, 4.5), (750, 4.4), (745, 4.4), (19, 4.4), (15, 4.4), (609, 4.4)]
549 [(20, 4.6), (92, 4.6), (19, 4.5), (124, 4.4), (151, 4.4), (1011, 4.4), (182, 4.3), (745, 4.3), (377, 4.2), (1007, 4.2)]
552 [(150, 4.8), (414, 4.8), (738, 4.8), (1008, 4.8), (745, 4.7), (737, 4.7), (1011, 4.7), (1007, 4.7), (716, 4.7), (695, 4.7)]
554 [(124, 4.7), (1011, 4.7), (737, 4.6), (20, 4.6), (92, 4.6), (750, 4.6), (93, 4.6), (1007, 4.5), (376, 4.5), (716, 4.5)]
556 [(20, 4.8), (415, 4.7), (150, 4.7), (124, 4.6), (92, 4.6), (414, 4.6), (737, 4.6), (745, 4.6), (15, 4.6), (750, 4.6)]
557 [(124, 4.8), (92, 4.8), (737, 4.6), (15, 4.6), (1008, 4.6), (695, 4.6), (151, 4.5), (453, 4.5), (170, 4.5), (544, 4.4)]
558 [(20, 4.8), (150, 4.7), (376, 4.6), (738, 4.6), (377, 4.5), (238, 4.5), (91, 4.5), (716, 4.5), (98, 4.5), (453, 4.5)]
559 [(150, 

1072 [(414, 4.8), (311, 4.8), (750, 4.8), (1008, 4.8), (1011, 4.7), (737, 4.7), (1007, 4.7), (558, 4.6), (156, 4.6), (1003, 4.6)]
1073 [(20, 5), (150, 4.9), (92, 4.9), (124, 4.9), (311, 4.9), (414, 4.9), (745, 4.8), (15, 4.8), (1008, 4.8), (750, 4.8)]
1074 [(20, 4.9), (750, 4.7), (1011, 4.7), (311, 4.6), (92, 4.6), (998, 4.6), (745, 4.6), (993, 4.6), (716, 4.6), (920, 4.5)]
1075 [(20, 5.0), (415, 4.9), (93, 4.8), (745, 4.8), (124, 4.8), (750, 4.8), (1011, 4.8), (738, 4.8), (322, 4.7), (156, 4.7)]
1076 [(150, 4.7), (20, 4.7), (124, 4.7), (414, 4.6), (745, 4.6), (415, 4.6), (737, 4.6), (1008, 4.6), (92, 4.5), (558, 4.5)]
1077 [(124, 4.8), (695, 4.7), (414, 4.7), (738, 4.7), (737, 4.6), (745, 4.6), (91, 4.6), (15, 4.6), (19, 4.6), (1008, 4.6)]
1078 [(415, 4.6), (20, 4.5), (414, 4.5), (311, 4.5), (124, 4.4), (15, 4.4), (716, 4.4), (1008, 4.4), (1011, 4.4), (19, 4.3)]
1079 [(124, 4.8), (415, 4.8), (20, 4.8), (745, 4.8), (414, 4.8), (750, 4.7), (92, 4.7), (716, 4.7), (753, 4.6), (993, 4.6)]


1583 [(1011, 4.8), (737, 4.8), (750, 4.7), (716, 4.7), (322, 4.7), (1008, 4.7), (745, 4.7), (997, 4.7), (311, 4.6), (979, 4.6)]
1584 [(20, 4.8), (415, 4.8), (150, 4.8), (10, 4.7), (738, 4.7), (716, 4.7), (737, 4.7), (92, 4.7), (124, 4.7), (745, 4.7)]
1585 [(20, 4.8), (150, 4.8), (15, 4.8), (124, 4.7), (311, 4.7), (415, 4.7), (1011, 4.7), (737, 4.6), (10, 4.6), (236, 4.6)]
1586 [(151, 4.9), (750, 4.8), (1008, 4.8), (236, 4.8), (238, 4.7), (91, 4.7), (1011, 4.7), (1007, 4.7), (609, 4.7), (377, 4.6)]
1587 [(150, 4.9), (124, 4.8), (20, 4.8), (737, 4.8), (415, 4.8), (414, 4.8), (92, 4.8), (750, 4.7), (745, 4.7), (377, 4.7)]
1588 [(150, 4.7), (20, 4.7), (92, 4.7), (377, 4.6), (376, 4.6), (695, 4.6), (414, 4.6), (1008, 4.6), (750, 4.5), (737, 4.5)]
1589 [(745, 4.3), (20, 4.2), (182, 4.2), (737, 4.2), (1008, 4.0), (558, 3.9), (716, 3.9), (152, 3.9), (1007, 3.9), (453, 3.8)]
1590 [(738, 4.4), (15, 4.3), (1008, 4.3), (1007, 4.2), (182, 4.1), (156, 4.1), (1446, 4.0), (995, 4.0), (920, 4.0), (544,

2101 [(150, 4.8), (415, 4.8), (20, 4.7), (414, 4.7), (92, 4.7), (15, 4.7), (124, 4.7), (745, 4.7), (750, 4.7), (311, 4.6)]
2102 [(152, 4.6), (716, 4.4), (920, 4.2), (679, 4.1), (947, 4.1), (132, 4.0), (1294, 3.9), (149, 3.6), (349, 3.5)]
2103 [(150, 5), (20, 5.0), (415, 4.9), (322, 4.9), (737, 4.9), (1011, 4.8), (997, 4.8), (92, 4.8), (750, 4.8), (738, 4.8)]
2104 [(15, 5), (96, 4.9), (20, 4.9), (738, 4.9), (737, 4.9), (415, 4.9), (414, 4.9), (745, 4.9), (609, 4.9), (750, 4.9)]
2105 [(20, 4.7), (377, 4.7), (415, 4.7), (92, 4.6), (993, 4.6), (745, 4.6), (124, 4.6), (1008, 4.5), (1011, 4.5), (737, 4.5)]
2106 [(20, 5), (150, 5), (745, 5.0), (415, 4.9), (92, 4.9), (609, 4.9), (124, 4.9), (93, 4.9), (737, 4.9), (414, 4.9)]
2107 [(745, 4.9), (92, 4.9), (124, 4.9), (695, 4.9), (151, 4.8), (1011, 4.8), (750, 4.8), (998, 4.8), (405, 4.8), (558, 4.7)]
2108 [(150, 4.9), (20, 4.9), (737, 4.9), (414, 4.8), (182, 4.8), (92, 4.8), (96, 4.8), (753, 4.8), (1008, 4.8), (716, 4.8)]
2110 [(20, 5.0), (150, 

...

24842 [(15, 4.7), (20, 4.7), (19, 4.7), (124, 4.7), (745, 4.7), (376, 4.6), (92, 4.6), (737, 4.6), (414, 4.6), (415, 4.6)]
24844 [(150, 4.9), (737, 4.8), (20, 4.8), (414, 4.8), (415, 4.8), (998, 4.8), (745, 4.8), (124, 4.8), (1011, 4.7), (753, 4.7)]
24846 [(20, 4.9), (753, 4.9), (745, 4.8), (737, 4.8), (10, 4.8), (738, 4.8), (414, 4.8), (1011, 4.8), (415, 4.7), (971, 4.7)]
24849 [(20, 4.8), (124, 4.7), (150, 4.7), (415, 4.7), (414, 4.6), (92, 4.6), (737, 4.6), (311, 4.6), (1011, 4.6), (745, 4.6)]
24853 [(150, 5.0), (182, 5.0), (415, 5.0), (737, 5.0), (20, 5.0), (998, 4.9), (151, 4.9), (376, 4.9), (414, 4.9), (745, 4.9)]
24855 [(20, 4.7), (150, 4.6), (415, 4.5), (92, 4.5), (737, 4.5), (738, 4.5), (311, 4.5), (124, 4.5), (19, 4.4), (93, 4.4)]
24856 [(415, 4.7), (92, 4.7), (745, 4.7), (20, 4.6), (695, 4.6), (971, 4.6), (376, 4.6), (322, 4.6), (998, 4.5), (1008, 4.5)]
24859 [(20, 4.7), (150, 4.6), (124, 4.6), (1008, 4.6), (737, 4.6), (415, 4.6), (151, 4.6), (414, 4.6), (92, 4.6), (716, 4.5

25686 [(20, 4.8), (150, 4.8), (151, 4.8), (92, 4.8), (414, 4.7), (1008, 4.7), (93, 4.7), (695, 4.7), (737, 4.7), (1446, 4.6)]
25688 [(20, 5.0), (150, 4.9), (92, 4.8), (415, 4.8), (738, 4.8), (96, 4.8), (124, 4.8), (19, 4.8), (695, 4.7), (745, 4.7)]
25689 [(150, 4.8), (414, 4.8), (124, 4.8), (377, 4.8), (415, 4.7), (20, 4.7), (376, 4.7), (92, 4.7), (609, 4.7), (750, 4.7)]
25690 [(150, 4.6), (20, 4.6), (1011, 4.5), (92, 4.5), (745, 4.5), (737, 4.4), (15, 4.4), (311, 4.4), (738, 4.4), (124, 4.4)]
25691 [(150, 4.6), (750, 4.6), (92, 4.5), (20, 4.5), (124, 4.5), (415, 4.5), (745, 4.5), (753, 4.5), (979, 4.5), (1011, 4.5)]
25705 [(20, 4.8), (150, 4.8), (1011, 4.8), (92, 4.7), (737, 4.7), (716, 4.7), (414, 4.7), (750, 4.7), (311, 4.7), (98, 4.7)]
25706 [(15, 5), (20, 5), (92, 5), (124, 5), (311, 5), (1008, 5.0), (415, 5.0), (750, 5.0), (414, 5.0), (93, 4.9)]
25707 [(1011, 4.7), (1008, 4.7), (20, 4.7), (414, 4.6), (695, 4.6), (93, 4.6), (182, 4.6), (311, 4.6), (376, 4.6), (124, 4.6)]
25709 [(1

26531 [(20, 4.7), (311, 4.7), (10, 4.6), (738, 4.6), (414, 4.6), (92, 4.6), (15, 4.6), (745, 4.5), (19, 4.5), (750, 4.5)]
26532 [(20, 4.9), (150, 4.8), (92, 4.8), (124, 4.7), (415, 4.7), (376, 4.7), (377, 4.7), (15, 4.7), (737, 4.7), (322, 4.7)]
26533 [(92, 5), (150, 5), (20, 4.9), (1011, 4.9), (415, 4.9), (414, 4.9), (745, 4.9), (19, 4.8), (124, 4.8), (311, 4.8)]
26536 [(20, 4.5), (737, 4.5), (124, 4.5), (19, 4.5), (322, 4.5), (1008, 4.5), (92, 4.5), (997, 4.4), (745, 4.4), (15, 4.4)]
26537 [(150, 4.8), (750, 4.8), (20, 4.7), (124, 4.7), (979, 4.7), (414, 4.7), (737, 4.7), (15, 4.7), (1008, 4.7), (738, 4.7)]
26539 [(311, 5.0), (124, 4.9), (20, 4.9), (96, 4.9), (1011, 4.9), (750, 4.8), (999, 4.8), (19, 4.8), (609, 4.8), (737, 4.8)]
26542 [(415, 4.8), (414, 4.8), (98, 4.8), (716, 4.7), (737, 4.7), (745, 4.7), (738, 4.7), (1008, 4.7), (558, 4.7), (405, 4.7)]
26544 [(1011, 4.8), (737, 4.8), (92, 4.8), (750, 4.8), (414, 4.8), (124, 4.8), (20, 4.8), (150, 4.8), (415, 4.8), (745, 4.7)]
26546

27332 [(150, 4.9), (695, 4.8), (414, 4.8), (415, 4.8), (124, 4.7), (20, 4.7), (92, 4.7), (93, 4.7), (745, 4.7), (1008, 4.7)]
27333 [(750, 4.5), (182, 4.5), (737, 4.4), (1008, 4.4), (716, 4.4), (757, 4.4), (745, 4.4), (93, 4.4), (1011, 4.4), (96, 4.4)]
27335 [(20, 5.0), (415, 4.9), (695, 4.9), (150, 4.9), (414, 4.8), (737, 4.8), (92, 4.8), (93, 4.8), (716, 4.8), (993, 4.8)]
27336 [(20, 4.9), (150, 4.8), (737, 4.8), (311, 4.8), (415, 4.8), (124, 4.8), (10, 4.8), (19, 4.7), (1011, 4.7), (414, 4.7)]
27338 [(415, 4.8), (150, 4.8), (20, 4.8), (738, 4.7), (414, 4.7), (92, 4.7), (609, 4.7), (737, 4.7), (753, 4.7), (124, 4.7)]
27339 [(20, 4.7), (124, 4.6), (150, 4.6), (414, 4.5), (745, 4.5), (695, 4.5), (10, 4.5), (92, 4.5), (19, 4.4), (738, 4.4)]
27340 [(753, 4.7), (20, 4.7), (415, 4.6), (150, 4.6), (745, 4.6), (92, 4.6), (414, 4.6), (1011, 4.5), (124, 4.5), (716, 4.5)]
27342 [(20, 4.9), (150, 4.9), (415, 4.9), (92, 4.9), (151, 4.8), (745, 4.8), (124, 4.7), (377, 4.7), (376, 4.7), (738, 4.7)]


28117 [(20, 4.8), (150, 4.7), (92, 4.6), (414, 4.6), (311, 4.6), (738, 4.6), (750, 4.6), (10, 4.6), (609, 4.6), (745, 4.5)]
28118 [(745, 5.0), (737, 4.9), (150, 4.9), (20, 4.8), (1008, 4.8), (414, 4.8), (750, 4.7), (92, 4.7), (10, 4.7), (19, 4.7)]
28121 [(20, 5), (19, 4.8), (124, 4.8), (738, 4.8), (15, 4.8), (997, 4.7), (104, 4.7), (93, 4.7), (96, 4.7), (695, 4.7)]
28122 [(20, 5), (415, 5.0), (92, 4.9), (1011, 4.9), (1008, 4.8), (414, 4.8), (737, 4.8), (93, 4.8), (738, 4.8), (609, 4.8)]
28124 [(20, 4.6), (92, 4.5), (19, 4.4), (1011, 4.4), (15, 4.4), (311, 4.4), (745, 4.4), (609, 4.4), (737, 4.4), (750, 4.3)]
28125 [(20, 4.9), (150, 4.9), (415, 4.9), (737, 4.9), (414, 4.9), (745, 4.9), (750, 4.8), (738, 4.8), (1011, 4.8), (92, 4.8)]
28127 [(92, 4.9), (415, 4.9), (150, 4.8), (414, 4.8), (311, 4.8), (609, 4.8), (737, 4.8), (93, 4.8), (750, 4.8), (15, 4.8)]
28128 [(20, 4.7), (1011, 4.7), (737, 4.7), (92, 4.6), (1008, 4.6), (124, 4.6), (716, 4.6), (376, 4.6), (322, 4.6), (997, 4.6)]
28129 [

### 整体命中率

In [76]:

def HitRate(topNPredicted, leftOutPredictions):
    hits = 0
    total = 0
 
    for leftOut in leftOutPredictions:
        userID = leftOut[0]
        leftOutMovieID = leftOut[1]
        
        hit = False
        for movieID, predictedRating in topNPredicted[int(userID)]:
            if (int(leftOutMovieID) == int(movieID)):
                hit = True
                break
        if (hit) :
            hits += 1

        total += 1

    return hits/total
print("整体命中率: ", HitRate(topNPredicted, leftOutPredictions))

整体命中率:  0.20156963596860727


### 评分命中率

In [80]:
def RatingHitRate(topNPredicted, leftOutPredictions):
    hits = defaultdict(float)
    total = defaultdict(float)    
    for userID, leftOutMovieID, actualRating, estimatedRating, _ in leftOutPredictions:
        
        hit = False
        for movieID, predictedRating in topNPredicted[int(userID)]:
            if (int(leftOutMovieID) == movieID):
                hit = True
                break
        if (hit) :
            hits[actualRating] += 1
        total[actualRating] += 1

    for rating in sorted(hits.keys()):
        print(rating, hits[rating] / total[rating])
print("评分的命中率: ")
RatingHitRate(topNPredicted, leftOutPredictions)

评分的命中率: 
1.0 0.06666666666666667
2.0 0.06976744186046512
3.0 0.09087495306045813
4.0 0.16397885892397343
5.0 0.25305280528052804


### 累积命中率 ###

In [88]:
def CumulativeHitRate(topNPredicted, leftOutPredictions, ratingCutoff=0):
    hits = 0
    total = 0

    for userID, leftOutMovieID, actualRating, estimatedRating, _ in leftOutPredictions:

        if (actualRating >= ratingCutoff):            
            hit = False
            for movieID, predictedRating in topNPredicted[int(userID)]:
                if (int(leftOutMovieID) == movieID):
                    hit = True
                    break
            if (hit) :
                hits += 1
            total += 1
    return hits/total
print("累积命中率 (rating >= 4): ", CumulativeHitRate(topNPredicted, leftOutPredictions, 4.0))

累积命中率 (rating >= 4):  0.21934458177342428


### 平均互惠命中排名 ###

In [91]:
def AverageReciprocalHitRank(topNPredicted, leftOutPredictions):
    summation = 0
    total = 0
        
    for userID, leftOutMovieID, actualRating, estimatedRating, _ in leftOutPredictions:
        
        hitRank = 0
        rank = 0
        for movieID, predictedRating in topNPredicted[int(userID)]:
            rank = rank + 1
            if (int(leftOutMovieID) == movieID):
                hitRank = rank
                break
        if (hitRank > 0) :
                summation += 1.0 / hitRank

        total += 1

    return summation / total

print("平均互惠命中排名: ", AverageReciprocalHitRank(topNPredicted, leftOutPredictions))

平均互惠命中排名:  0.06899543157247194
